In [153]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
import numpy as np

In [154]:
sm_boto = boto3.client("sagemaker")
boto_session = boto3.Session(region_name="us-east-1")
sess = sagemaker.Session(boto_session=boto_session)
# sess = sagemaker.Session()
# region = sess.boto_session.region_name
# sess = sagemaker.Session(boto_session=boto3.Session(region_name="ap-southeast-1"))
bucket = "mobawsbucket"
print("Using bucket - ", bucket)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Using bucket -  mobawsbucket


In [155]:
df = pd.read_csv("train.csv")

In [156]:
df.shape

(2000, 21)

In [157]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [158]:
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [159]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [160]:
df.isnull().mean() * 100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [161]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [162]:
label = features.pop(-1)
label

'price_range'

In [163]:
x = df[features]
y = df[label]

In [164]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [165]:
y.head() # 0 - low risk, 1 - high risk

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [166]:
x.shape

(2000, 20)

In [167]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [168]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size=0.15, random_state=0)

In [169]:
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(1700, 20) (1700,) (300, 20) (300,)


In [170]:
trainX = pd.DataFrame(X_train)
trainX[label] = Y_train

testX = pd.DataFrame(X_test)
testX[label] = Y_test

In [171]:
print(trainX.shape, testX.shape)

(1700, 21) (300, 21)


In [172]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,...,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,...,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,...,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,...,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,...,48,1012,959,17,7,6,0,1,0,0


In [173]:
testX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [174]:
trainX.to_csv("train_V1.csv", index=False)
testX.to_csv("test_V1.csv", index=False)

In [214]:
sk_prefix = "sagemaker/mobile_price_classification/sklearncontainer"

trainpath = sess.upload_data(path="train_V1.csv", bucket=bucket, key_prefix=sk_prefix)
testpath = sess.upload_data(path="test_V1.csv", bucket=bucket, key_prefix=sk_prefix)

print(trainpath)
print(testpath)

s3://mobawsbucket/sagemaker/mobile_price_classification/sklearncontainer/train_V1.csv
s3://mobawsbucket/sagemaker/mobile_price_classification/sklearncontainer/test_V1.csv


In [222]:
%%writefile script.py

import argparse
import os
import joblib
import pandas as pd
import boto3
import pathlib
from io import StringIO
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score

# SageMaker requires a model_fn to load the model for inference
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ == '__main__':
    print("Extracting arguments")
    parser = argparse.ArgumentParser()

    # Define hyperparameters and input data paths as arguments
    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--random_state', type=int, default=0)

    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default='/opt/ml/input/data/train')
    parser.add_argument('--test', type=str, default='/opt/ml/input/data/test')
    parser.add_argument('--train_file', type=str, default='train_V1.csv')
    parser.add_argument('--test_file', type=str, default='test_V1.csv')


    args, _ = parser.parse_known_args()

    # Load data
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    # Separate features and target
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    # Split data into training and testing sets
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the Random Forest Classifier
    clf = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state)
    clf.fit(X_train, y_train)

    # Evaluate the model (optional)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    test_rep = classification_report(y_test, y_pred)
    print(f"Model Accuracy: {accuracy}")
    print("Classification report: ")
    print(test_rep)

    # Save the trained model
    joblib.dump(clf, os.path.join(args.model_dir, "model.joblib"))

Overwriting script.py


In [223]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(entry_point="script.py",
                            role="arn:aws:iam::380860201580:role/awsSageMaker",
                            instance_count=1,
                            instance_type="ml.m5.xlarge",
                            framework_version=FRAMEWORK_VERSION,
                            base_job_name="RF-customer-sklearn",
                            hyperparameters={
                                "n_estimators":100,
                                "random_state":0,
                            },
                            use_spot_instances = True,
                            max_wait=7200,
                            max_run=3600
                            )

In [224]:
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: RF-customer-sklearn-2025-10-28-16-14-11-526


2025-10-28 16:14:18 Starting - Starting the training job...
2025-10-28 16:14:54 Downloading - Downloading input data...
2025-10-28 16:15:14 Downloading - Downloading the training image...
2025-10-28 16:16:01 Training - Training image download completed. Training in progress.
2025-10-28 16:16:01 Uploading - Uploading generated training model.2025-10-28 16:15:56,258 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-10-28 16:15:56,261 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-10-28 16:15:56,308 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-10-28 16:15:56,527 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-10-28 16:15:56,540 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-10-28 16:15:56,553 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-10-28 16:15:5

In [225]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto.describe_training_job(TrainingJobName=sklearn_estimator.latest_training_job.name)["ModelArtifacts"]["S3ModelArtifacts"]
print(artifact)


2025-10-28 16:16:13 Starting - Preparing the instances for training
2025-10-28 16:16:13 Downloading - Downloading the training image
2025-10-28 16:16:13 Training - Training image download completed. Training in progress.
2025-10-28 16:16:13 Uploading - Uploading generated training model
2025-10-28 16:16:13 Completed - Training job completed
s3://sagemaker-us-east-1-380860201580/RF-customer-sklearn-2025-10-28-16-14-11-526/output/model.tar.gz


In [ ]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model = SKLearnModel(name = model_name, 
                     model_data=artifact,
                     role="arn:aws:iam::380860201580:role/awsSageMaker",
                     entry_point="script.py",
                     framework_version=FRAMEWORK_VERSION)

In [227]:
model

In [228]:
#Endpoint deployment
endpoint_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint name={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)

Endpoint name=Custom-sklearn-model2025-10-28-16-25-31


INFO:sagemaker:Creating model with name: Custom-sklearn-model2025-10-28-16-23-33
INFO:sagemaker:Creating endpoint-config with name Custom-sklearn-model2025-10-28-16-25-31
INFO:sagemaker:Creating endpoint with name Custom-sklearn-model2025-10-28-16-25-31


------!

In [229]:
predictor

In [230]:
predictor.predict(testX[features][0:2].values.tolist())

array([3, 0])

In [233]:
sm_boto.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '41853daf-41a3-425e-8644-c333cf4f2c5d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '41853daf-41a3-425e-8644-c333cf4f2c5d',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 28 Oct 2025 16:31:55 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}